In [14]:
import os
import collections
import time
import pandas as pd
import nglview as nv
from rdkit import Chem, RDConfig
from rdkit.Chem import Draw
from rdkit.Chem import AllChem

In [15]:
mol1 = 'dataset/F059-1017_docked.sdf'
mol2 = 'dataset/E966-0578_docked.sdf'
hit = 'dataset/CDK8_ML_Hit.sdf'

In [16]:
supplier = Chem.SDMolSupplier(mol1) # this is an iterator
mol_1 = supplier[0]

supplier = Chem.SDMolSupplier(mol2) # this is an iterator
mol_2 = supplier[0]

mols = [mol_1, mol_2]

# if mol:
#     img = Draw.MolToImage(mol)


In [17]:
feature_factory = AllChem.BuildFeatureFactory(os.path.join(RDConfig.RDDataDir,'BaseFeatures.fdef'))
feature_factory.GetFeatureFamilies()

('Donor',
 'Acceptor',
 'NegIonizable',
 'PosIonizable',
 'ZnBinder',
 'Aromatic',
 'Hydrophobe',
 'LumpedHydrophobe')

In [18]:
features = feature_factory.GetFeaturesForMol(mols[0])
print(f"Number of features: {len(features)}")

Number of features: 13


In [19]:
features[0].GetFamily()

'Donor'

In [20]:
feature_frequency = collections.Counter(sorted([feature.GetFamily() for feature in features]))
feature_frequency

Counter({'Hydrophobe': 4,
         'Acceptor': 3,
         'LumpedHydrophobe': 3,
         'Aromatic': 2,
         'Donor': 1})

In [21]:
molecule_feature_frequencies = []
for mol in mols:
    features = [feature.GetFamily() for feature in feature_factory.GetFeaturesForMol(mol)]
    feature_frequency = collections.Counter(features)
    molecule_feature_frequencies.append(feature_frequency)

feature_frequencies_df = (
    pd.DataFrame(
        molecule_feature_frequencies,
        index=[f"Mol{i}" for i, _ in enumerate(mols, 1)],
    )
    .fillna(0)
    .astype(int)
)
feature_frequencies_df.transpose()

,Mol1,Mol2
Donor,1,1
Acceptor,3,4
Aromatic,2,2
Hydrophobe,4,5
LumpedHydrophobe,3,2


In [22]:
acceptors = []
donors = []
hydrophobics = []

for mol in mols:
    acceptors.append(feature_factory.GetFeaturesForMol(mol, includeOnly="Acceptor"))
    donors.append(feature_factory.GetFeaturesForMol(mol, includeOnly="Donor"))
    hydrophobics.append(feature_factory.GetFeaturesForMol(mol, includeOnly="Hydrophobe"))

features = {
    "donors": donors,
    "acceptors": acceptors,
    "hydrophobics": hydrophobics,
}

In [23]:
feature_colors = {
    "donors": (0, 0.9, 0),  # Green
    "acceptors": (0.9, 0, 0),  # Red
    "hydrophobics": (1, 0.9, 0),  # Yellow
}

In [30]:
pml_file = 'features1.pml'
load_sdf = mol1

def colorToHex(rgb):
    return f"[{int(255 * rgb[0])},{int(255 * rgb[1])},{int(255 * rgb[2])}]"

with open(pml_file, "w") as f:
    f.write(f"load {load_sdf}\n")  # Load the molecule in PyMOL
    features = feature_factory.GetFeaturesForMol(mol_1)
    print(len(features))
    # print(features)
    for i, feat in enumerate(features):
        type = feat.GetFamily()
        if type == "Acceptor":
            pos = feat.GetPos()  # Feature position
            color_hex = colorToHex((0.9, 0, 0))  # Convert to PyMOL-compatible color format
            f.write(
                f"pseudoatom feature_{i}, pos=[{pos.x}, {pos.y}, {pos.z}], color=red\n"
            )
        if type == "Donor":
            pos = feat.GetPos()  # Feature position

            color_hex = (0, 0.9, 0)  # Convert to PyMOL-compatible color format
            f.write(
                f"pseudoatom feature_{i}, pos=[{pos.x}, {pos.y}, {pos.z}], color=blue\n"
            )
        if type == "Hydrophobe":
            pos = feat.GetPos()
            f.write(f"pseudoatom feature_{i}, pos=[{pos.x}, {pos.y}, {pos.z}], color=green\n"
                    )
        if type == 'Aromatic':
            pos = feat.GetPos()
            f.write(f"pseudoatom feature_{i}, pos=[{pos.x}, {pos.y}, {pos.z}], color=pink\n"
                    )
        if type == 'LumpedHydrophobe':
            pos = feat.GetPos()
            f.write(f"pseudoatom feature_{i}, pos=[{pos.x}, {pos.y}, {pos.z}], color=green\n"
                    )
    f.write("show spheres, feature_*\n")
    f.write("set sphere_scale, 0.5\n")  # Adjust sphere size in PyMOL
print(f"Feature visualization script written to {pml_file}.")

13
Feature visualization script written to features1.pml.


In [11]:
def show_ligands(molecules):
    """Generate a view of the ligand molecules.

    Parameters
    -----------
    molecules: list of rdkit.Chem.rdchem.Mol

    Returns
    ----------
    nglview.widget.NGLWidget
    """
    view = nv.NGLWidget()
    for molecule in molecules:
        component = view.add_component(molecule)
        time.sleep(0.1)
        component.clear()
        component.add_ball_and_stick(multipleBond=True)
    return view

In [12]:
def visualize_features(
    molecules,
    features,
    feature_type="features",
    color="yellow",
    sphere_radius=0.5,
):
    """Generate a view of the molecules highlighting the specified feature type.

    Parameters
    -----------
    molecules: list of rdkit.Chem.rdchem.Mol
        molecules to be visualized
    features: list of tuples of rdkit.Chem.rdMolChemicalFeatures.MolChemicalFeature
        extracted features from molecules chosen to be highlighted
    feature_type: string, optional
        name of the feature to be highlighted
    color: string, optional
        color used to display the highlighted features
    sphere_radius: float, optional
        display size of the highlighted features

    Returns
    ----------
    nglview.widget.NGLWidget
    """
    print(f"Number of {feature_type} in all ligands: {sum([len(i) for i in features])}")
    view = show_ligands(molecules)
    for i, feature_set in enumerate(features, 1):
        for feature in feature_set:
            loc = list(feature.GetPos())
            label = f"{feature_type}_{i}"
            view.shape.add_sphere(loc, color, sphere_radius, label)
    return view

In [13]:
view = show_ligands(mols)
view.render_image(trim=True, factor=2, transparent=True);
view

NGLWidget()